In [1]:
import pandas as pd

df = pd.read_csv("/content/1763814328_3.csv")
df.head()


,_time,host,sourcetype,source,src,dest,user,signature,action,_raw
0,2018-08-20T20:26:46.000+0500,serverless,aws:cloudwatchlogs:vpcflow,lambda:vpcflowlogs:us-west-1,NaN,NaN,NaN,NaN,NaN,2 622676721278 eni-0536faba73134a9b7 13.125.33...
1,2018-08-20T20:26:46.000+0500,serverless,aws:cloudwatchlogs:vpcflow,lambda:vpcflowlogs:us-west-1,NaN,NaN,NaN,NaN,NaN,2 622676721278 eni-0536faba73134a9b7 172.16.0....
2,2018-08-20T20:22:46.000+0500,serverless,aws:cloudwatchlogs:vpcflow,lambda:vpcflowlogs:us-west-1,NaN,NaN,NaN,NaN,NaN,2 622676721278 eni-0536faba73134a9b7 172.16.0....
3,2018-08-20T20:22:46.000+0500,serverless,aws:cloudwatchlogs:vpcflow,lambda:vpcflowlogs:us-west-1,NaN,NaN,NaN,NaN,NaN,2 622676721278 eni-0536faba73134a9b7 13.125.33...
4,2018-08-20T20:20:46.000+0500,serverless,aws:cloudwatchlogs:vpcflow,lambda:vpcflowlogs:us-west-1,NaN,NaN,NaN,NaN,NaN,2 622676721278 eni-0536faba73134a9b7 13.125.33...


In [2]:
TEXT_FIELDS = ["_time", "host", "sourcetype", "source", "src", "dest",
               "user", "signature", "action", "_raw"]

def row_to_text(row):
    parts = []
    for col in TEXT_FIELDS:
        if col in row and pd.notna(row[col]):
            parts.append(f"{col}={row[col]}")
    return " | ".join(parts)

df["doc_text"] = df.apply(row_to_text, axis=1)
docs = df["doc_text"].tolist()
len(docs)


5000

In [3]:
!pip install faiss-cpu langchain langchain-community sentence-transformers groq pandas

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# LIMIT FOR PROTOTYPE (adjust as you like)
MAX_DOCS = 20000   # try 5k–20k first to get it running

docs_small = docs[:MAX_DOCS]
print("Using", len(docs_small), "logs for the index.")

emb_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=emb_model_name)

Using 5000 logs for the index.


/tmp/ipython-input-389357726.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=emb_model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from tqdm import tqdm  # optional progress bar
import math

batch_size = 512
faiss_store = None

num_docs = len(docs_small)
num_batches = math.ceil(num_docs / batch_size)
print("Building FAISS index in", num_batches, "batches...")

for i in tqdm(range(0, num_docs, batch_size)):
    batch = docs_small[i:i+batch_size]

    if faiss_store is None:
        faiss_store = FAISS.from_texts(batch, embedding)
    else:
        faiss_store.add_texts(batch)

print("Index built!")
faiss_store.save_local("botsv3_faiss_index")


Building FAISS index in 10 batches...


100%|██████████| 10/10 [08:51<00:00, 53.12s/it]

Index built!


In [7]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "[REDACTED_GROQ_KEY]"

client = Groq(api_key=os.environ["GROQ_API_KEY"])
MODEL_ID = "llama-3.1-8b-instant"


In [8]:
def groq_chat(system_prompt, user_prompt):
    resp = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt},
        ],
        temperature=0.1,
    )
    return resp.choices[0].message.content


In [9]:
def retrieve_context(query, k=5):
    results = faiss_store.similarity_search(query, k=k)
    return "\n\n".join([r.page_content for r in results])

def analyze_log_with_rag(log_text, k=8):
    # 1) Get similar historical logs as context
    context = retrieve_context(log_text, k=k)

    system_prompt = """
You are a cybersecurity analyst AI working in a SOC.
You receive a single log event and a set of similar historical events (context).
Decide if the new event is malicious, suspicious, or benign.
Respond in compact JSON with keys:
- classification: "malicious" | "suspicious" | "benign"
- severity: integer 0-100
- confidence: integer 0-100
- reason: short explanation
- suggested_action: short recommendation
Do not add extra text around the JSON.
"""

    user_prompt = f"""
NEW_EVENT:
{log_text}

CONTEXT_LOGS:
{context}
"""

    return groq_chat(system_prompt, user_prompt)


In [10]:
sample_log = df["doc_text"].iloc[0]
print(sample_log[:400])
result_json = analyze_log_with_rag(sample_log)
print(result_json)

_time=2018-08-20T20:26:46.000+0500 | host=serverless | sourcetype=aws:cloudwatchlogs:vpcflow | source=lambda:vpcflowlogs:us-west-1 | _raw=2 622676721278 eni-0536faba73134a9b7 13.125.33.130 172.16.0.178 40396 11211 17 1 50 1534778806 1534778866 ACCEPT OK
{
  "classification": "suspicious",
  "severity": 80,
  "confidence": 90,
  "reason": "Repeated traffic from 13.125.33.130 to 172.16.0.178 within a short time frame, which may indicate a potential scanning or probing activity.",
  "suggested_action": "Investigate the source IP 13.125.33.130 and the destination IP 172.16.0.178 for potential security threats."
}
